In [ ]:
# install openmodelica from its repository

shell_call <- function(command, ...) { # required for later call of shell...
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

shell_call('for deb in deb deb-src; do echo "$deb http://build.openmodelica.org/apt `lsb_release -cs` release"; done | sudo tee /etc/apt/sources.list.d/openmodelica.list')
shell_call('wget -q http://build.openmodelica.org/apt/openmodelica.asc -O- | sudo apt-key add -')
shell_call('apt update && apt install openmodelica')

# check 'omc' command
shell_call('omc --help')

In [ ]:
# install Funz through devtools
devtools::install_github('Funz/Funz.R')
library(Funz)

In [ ]:
# install plugin to support Modelica I/O
install.Model('Modelica', edit.script=FALSE)

In [4]:
# setup main input file
writeLines(con='NewtonCooling.mo',text=c(
'// @ref http://book.xogeny.com/behavior/equations/physical/',
'model NewtonCooling "An example of Newton s law of cooling"',
'  parameter Real T_inf=25 "Ambient temperature";',
'  parameter Real T0=90 "Initial temperature";',
'  parameter Real h=0.7 "Convective cooling coefficient";',
'  parameter Real A=1.0 "Surface area";',
'  parameter Real m=0.1 "Mass of thermal capacitance";',
'  parameter Real c_p=1.2 "Specific heat";',
'  Real T "Temperature";',
'initial equation',
'  T = T0 "Specify initial value for T";',
'equation',
'  m*c_p*der(T) = h*A*(T_inf-T) "Newton s law of cooling";',
'end NewtonCooling;'
))

In [ ]:
# wake up the Funz 'daemon' which will provide calculation services:
startCalculators(1)

In [ ]:
# run this file with Funz (that will check everything is ok for later parametric study)
Run(model="Modelica",input.files="NewtonCooling.mo")

In [7]:
# setup main parametrized (with `$convection`) input file
writeLines(con='NewtonCooling.mo.par',text=c(
'// @ref http://book.xogeny.com/behavior/equations/physical/',
'model NewtonCooling "An example of Newton s law of cooling"',
'  parameter Real T_inf=25 "Ambient temperature";',
'  parameter Real T0=90 "Initial temperature";',
'  parameter Real h=$convection "Convective cooling coefficient";',
'  parameter Real A=1.0 "Surface area";',
'  parameter Real m=0.1 "Mass of thermal capacitance";',
'  parameter Real c_p=1.2 "Specific heat";',
'  Real T "Temperature";',
'initial equation',
'  T = T0 "Specify initial value for T";',
'equation',
'  m*c_p*der(T) = h*A*(T_inf-T) "Newton s law of cooling";',
'end NewtonCooling;'
))

In [ ]:
# launch 6 calculations for different `convection` values (0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
Run(model="Modelica",input.files="NewtonCooling.mo.par", input.variables=list(convection=c(0.5,0.6,0.7,0.8,0.9,1.0)), output.expressions="min(T)")

In [ ]:
# install Brent root finding algorithm
install.Design('Brent')

In [ ]:
# find the `convection` value leading to `min(T) = 25.2` (with relative precision of 0.01 on `convection` value)
RunDesign(model="Modelica",input.files="NewtonCooling.mo.par", input.variables=list(convection="[0.5,1.0]"), output.expressions="min(T)", 
          design="Brent", design.options=list(ytarget=25.2, ytol=0.01))